# Standardized Linear Regression Model

In [0]:
# ===
# Linear Regression (PySpark + MLlib)
# Handles high-VIF feature removal + hyperparameter tuning + RMSE + coefficient summary
# ===

# Step 1: Install and import dependencies
!pip install pyspark matplotlib
# Install Java 11
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# Step 1.5: Load libraries
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml import Pipeline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Step 1.5: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Start Spark session
spark = SparkSession.builder.appName("LinearRegressionTuning").getOrCreate()

# Step 3: Load dataset
data_path = "/content/drive/MyDrive/ait614_rutting2/data/processed/rutting_climate_traffic.csv"
df = spark.read.csv(data_path, header=True, inferSchema=True)
print(f"Total rows: {df.count()}")
df.printSchema()

# Step 4: Define features and target
# Removed features with VIF > 5
removed_features = [
    "PRECIP_DAYS",
    "REL_HUM_AVG_AVG",
    "SHORTWAVE_SURFACE_AVG",
    "TEMP_AVG",
    "EVAPORATION",
    "AADTT_VEH_CLASS_9_TREND",
    "AADTT_VEH_CLASS_11_TREND",
    "AADTT_VEH_CLASS_12_TREND"
]

all_features = [
    "REL_HUM_AVG_AVG",
    "PRECIPITATION",
    "EVAPORATION",
    "PRECIP_DAYS",
    "CLOUD_COVER_AVG",
    "SHORTWAVE_SURFACE_AVG",
    "TEMP_AVG",
    "FREEZE_INDEX",
    "FREEZE_THAW",
    "WIND_VELOCITY_AVG",
    "AADTT_VEH_CLASS_4_TREND",
    "AADTT_VEH_CLASS_5_TREND",
    "AADTT_VEH_CLASS_6_TREND",
    "AADTT_VEH_CLASS_7_TREND",
    "AADTT_VEH_CLASS_8_TREND",
    "AADTT_VEH_CLASS_9_TREND",
    "AADTT_VEH_CLASS_10_TREND",
    "AADTT_VEH_CLASS_11_TREND",
    "AADTT_VEH_CLASS_12_TREND",
    "AADTT_VEH_CLASS_13_TREND"
]

# Keep only features with acceptable VIF
features = [f for f in all_features if f not in removed_features]
target = "MAX_MEAN_DEPTH_1_8"

print("Features used (after removing high VIF):")
print(features)

# Step 5: Split data into train/test (80/20)
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)
print(f"Training set size: {train_df.count()}")
print(f"Test set size: {test_df.count()}")
print(f"Number of features: {len(features)}")
print(f"Number of targets: {len([target])}")

# Step 6: Assemble features and standardize
# VectorAssembler combines all feature columns into a single feature vector
assembler = VectorAssembler(inputCols=features, outputCol="features_raw")
# StandardScaler standardizes features to have zero mean and unit variance
scaler = StandardScaler(inputCol="features_raw", outputCol="features", withStd=True, withMean=True)

# Step 7: Define Linear Regression model
# Standardized linear regression; label column = target, features column = "features"
lr = LinearRegression(
    featuresCol="features",
    labelCol=target,
    predictionCol="prediction"
)

# Step 8: Create a pipeline
# Pipeline ensures sequential execution: assemble -> scale -> standardized linear regression
pipeline = Pipeline(stages=[assembler, scaler, lr])

# Step 9: Define hyperparameter grid
# Tune regularization (ridge/lasso strength) and elasticNetParam
# The grid tunes both regularization strength (regParam) and mixing type (elasticNetParam):
# - regParam = 0.0 -> disables regularization entirely (no ridge, no lasso) and fits normal standardized linear regression
# - regParam > 0.0 and elasticNetParam = 0.0 -> Ridge regression (L2)
# - regParam > 0.0 and elasticNetParam = 1.0 -> Lasso regression (L1)
# - regParam > 0.0 and 0 < elasticNetParam < 1 -> Elastic Net (mix of L1 & L2)
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.0, 0.01, 0.1, 0.3, 0.5])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])  # 0=ridge, 1=lasso
             .build())

# Step 10: Define evaluator (RMSE)
# RMSE is standard for regression evaluation
evaluator = RegressionEvaluator(
    labelCol=target,
    predictionCol="prediction",
    metricName="rmse"
)

# Step 11: Cross-validator for 5-fold CV
# CrossValidator performs hyperparameter tuning and returns best model
cv = CrossValidator(
    estimator=pipeline,          # pipeline as estimator
    estimatorParamMaps=paramGrid,# grid of hyperparameters
    evaluator=evaluator,         # RMSE evaluator
    numFolds=5,                  # 5-fold cross-validation
    parallelism=2,               # parallel threads
    seed=42
)

print("Training Linear Regression model with 5-fold CV...")
cv_model = cv.fit(train_df)  # Fit the pipeline with CV

# Step 12: Get 5-fold CV RMSE for best model configuration
best_cv_rmse = min(cv_model.avgMetrics)  # avg RMSE across folds for best hyperparameters
print(f"\n=== Linear Regression Results ===")
print(f"5-Fold CV RMSE (best hyperparameters): {best_cv_rmse:.3f}")

# Step 13: Print best hyperparameters
# The best hyperparameters are in the last stage of the pipeline (the LR model)
best_pipeline_model = cv_model.bestModel
best_lr_model = best_pipeline_model.stages[-1]
print("\n=== Best Hyperparameters ===")
print(f"Regularization Parameter (regParam): {best_lr_model._java_obj.getRegParam()}")
print(f"Elastic Net Param (elasticNetParam): {best_lr_model._java_obj.getElasticNetParam()}")

# Step 14: Coefficient summary with standardized coefficients
coefficients = best_lr_model.coefficients
intercept = best_lr_model.intercept
feature_names = features

# Create DataFrame for easy sorting and visualization
coef_data = {
    "Feature": feature_names,
    "Coefficient": coefficients,
    "AbsCoefficient": np.abs(coefficients)  # absolute value for magnitude
}

coef_df = pd.DataFrame(coef_data)
# Sort by absolute coefficient magnitude descending
coef_df = coef_df.sort_values(by="AbsCoefficient", ascending=False).reset_index(drop=True)
print("\n=== Standardized Coefficients ===")
print(coef_df[["Feature", "Coefficient"]])

# Step 15: Visualize standardized coefficients
plt.figure(figsize=(10, 6))
plt.barh(coef_df["Feature"], coef_df["AbsCoefficient"], color="steelblue")
plt.gca().invert_yaxis()
plt.title("Standardized Linear Regression Coefficient Magnitudes")
plt.xlabel("|Coefficient| (Magnitude)")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()

# Step 16: Stop Spark session
spark.stop()